In [1]:
# The following notebook uses the package extremes to fit a GEV model to
# series of annual maximum rainfall at multiple locations.
# Input data format: annual maximum series for each location by columns.
# Length of annual maximum series does not have to be the same for each location,
# however, if the AMS come from a gridded data source, all columns should have the same
# number of years and annual maxima.

# To install the library extremes and other required libraries in this notebook,
# uncomment the lines below and run the cell.

# install.packages('extrEmes')
# install.packages('glue')

In [2]:
library(extRemes)
library(glue)

Loading required package: Lmoments
Loading required package: distillery

Attaching package: ‘extRemes’

The following objects are masked from ‘package:stats’:

    qqnorm, qqplot



In [3]:
# Define input and output data files:
infile <- "output/historical_ches_AMS.csv"
outfile <- "output/historical_ches_GEV_params.csv"

In [29]:
# Read data into the notebook (might take some time because of file size):
ams <- read.csv(file=infile, header=T, sep=",")

In [18]:
# Check the size of the dataframe: number of years, length of AMS and number of grids
print(glue("Number of years: {nrow(ams)}"))
print(glue("Number of grids: {ncol(ams)}"))


Number of years: 68
Number of grids: 45955


In [38]:
# Need to set 9.96921e+36 values to NA. This value is inherited from the netCDF file.
navalue = 9.96921e+36
ams[ams == navalue] <- NA

In [40]:
# Remove columns where there all values are NA.
ams <- ams[colSums(!is.na(ams)) > 0]

In [41]:
print(glue("Number of grids after removing NA values: {ncol(ams)}"))

Number of grids after removing NA values: 35518


In [73]:
# Setting up the data in the required format by the library extremes, and
# creating the output matrix that will be saved as a csv in the outfile path specified above

model.name <- colnames(ams)
f = (length(model.name))
model.name <- colnames(ams)[c(2:f)]
rl.cals <- data.frame(matrix(ncol = 10, nrow = length(model.name)))
colnames(rl.cals) <- c("models", "L_loc", "loc", "U_loc",
                             "L_scale", "scale", "U_scale",
                             "L_shape", "shape", "U_shape")
rl.cals[1] <- as.vector(model.name)

In [74]:
# We will populate this matrix with the estimated parameters from the fit to each row,
# which corresponds to each grid in the gridded dataset.

rl.cals

models,L_loc,loc,U_loc,L_scale,scale,U_scale,L_shape,shape,U_shape
id_0_70,NA,NA,NA,NA,NA,NA,NA,NA,NA
id_0_71,NA,NA,NA,NA,NA,NA,NA,NA,NA
id_0_72,NA,NA,NA,NA,NA,NA,NA,NA,NA
id_0_73,NA,NA,NA,NA,NA,NA,NA,NA,NA
id_0_74,NA,NA,NA,NA,NA,NA,NA,NA,NA
id_0_75,NA,NA,NA,NA,NA,NA,NA,NA,NA
id_0_76,NA,NA,NA,NA,NA,NA,NA,NA,NA
id_0_77,NA,NA,NA,NA,NA,NA,NA,NA,NA
id_0_78,NA,NA,NA,NA,NA,NA,NA,NA,NA
id_0_79,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [75]:
# This for loop takes each column and tries to fit a GEV distribution to the,
# the AMS data using Maximum Likelihood Estimation method. 
# The confidence interval used to compute the upper and lower bounds of the,
# parameters is specified via the alpha parameters in the function call. The default
# method to compute the bounds in the extremes library is via bootstrapping.
# If the fitting method failed, the cell id will be printed.

# Again, might take some time due to the amount of data.

for (model in 2:f) {
    md = as.vector(ams[, (model)])
    id_ <- model.name[model-1]
    tryCatch({
        fit_mle <- fevd(md, method='MLE', type='GEV')

        param.loc <- as.vector(ci(fit_mle, alpha=0.05, type='parameter')[1,])
        param.scale <- as.vector(ci(fit_mle, alpha=0.05, type='parameter')[2,])
        param.shape <- as.vector(ci(fit_mle, alpha=0.05, type='parameter')[3,])

        rl.cals.single <- c(param.loc,param.scale, param.shape)
        rl.cals[model-1, c(2:10)] <- rl.cals.single},
        error = function(e){ message(" ", glue('{id_},'))
        })
    }

 id_19_135,
 id_19_136,
 id_20_128,
 id_20_136,
 id_20_137,
 id_20_138,
 id_21_128,
 id_21_132,
 id_21_133,
 id_21_135,
 id_21_136,
 id_21_137,
 id_21_138,
 id_21_139,
 id_22_128,
 id_22_129,
 id_22_130,
 id_22_132,
 id_22_133,
 id_22_134,
 id_22_135,
 id_22_136,
 id_22_137,
 id_22_138,
 id_22_139,
 id_22_141,
 id_23_128,
 id_23_129,
 id_23_130,
 id_23_131,
 id_23_132,
 id_23_133,
 id_23_134,
 id_23_135,
 id_23_136,
 id_23_137,
 id_23_138,
 id_23_139,
 id_23_140,
 id_23_141,
 id_24_127,
 id_24_128,
 id_24_129,
 id_24_130,
 id_24_131,
 id_24_132,
 id_24_133,
 id_24_134,
 id_24_135,
 id_24_136,
 id_24_137,
 id_24_138,
 id_24_139,
 id_24_140,
 id_24_141,
 id_24_142,
 id_24_145,
 id_25_128,
 id_25_129,
 id_25_130,
 id_25_131,
 id_25_132,
 id_25_133,
 id_25_134,
 id_25_135,
 id_25_136,
 id_25_137,
 id_25_138,
 id_25_139,
 id_25_140,
 id_25_141,
 id_25_142,
 id_25_143,
 id_25_144,
 id_25_145,
 id_25_146,
 id_25_150,
 id_25_151,
 id_26_45,
 id_26_128,
 id_26_129,
 id_26_130,
 id_26_131,
 id_2

In [77]:
md

[1]  86.40000  68.73502  63.25206 103.39506 100.13401  94.15682  66.20010
 [8] 125.27657  53.54694  51.76402 156.65433  78.70000 107.22871  86.52935
[15]  87.43301  93.55682  64.82839  99.27243  70.56837  64.18102  89.68070
[22]  67.49155  60.50000  59.79735  71.83680  52.70713  83.10000  82.78722
[29] 108.70000  46.12933  76.70000 105.36835  81.42659  60.60027  64.80000
[36]  55.90000 141.48610  51.45636  48.44935  55.89986 146.70723  81.30000
[43]  50.27177  66.39777  85.57703  66.39808 143.68674 150.04326 132.41116
[50]  45.24787 196.02634 174.27242 114.80000  87.71314  52.79981  86.40000
[57]  48.74144 171.87546 112.81235  61.88350 107.40000  82.80975 187.19064
[64] 183.86601  90.90000  71.49013  74.23332 182.19930

In [78]:
# There are approximately 456 grids out of 35518 grids where GEV model could not
# be fit. Investigate source of error. Fit failed for grid id_19_235:

ams[,"id_19_135"]

[1]  48.61150  55.68861  61.08456        NA 163.80000  25.39237  73.01843
 [8]  81.64407  50.74251  49.81351  49.19440  65.00000  36.72229  79.40552
[15]  73.88936  56.52803  43.42419  25.76526  63.59452  68.70727  34.70060
[22]  41.06218  55.12795  40.73146  97.61243  36.19745  28.25256  74.71206
[29]  85.48002  59.07370  67.80000  37.86177  46.13566  41.50352  51.07238
[36]  67.93443  38.90000  50.53236  33.72018  36.71457  39.94375  31.49413
[43]  56.48299  31.39092  42.55207  62.65078  55.85873  41.48393  88.40883
[50]  46.77733  44.03069  75.52331  52.37517 125.82433  53.72242  49.14603
[57]  73.72624  50.49666  39.90000  45.04666  42.54545  32.07519  98.59042
[64] 111.79226  31.20807  95.46149  68.38300  63.54851

In [81]:
# The fourth element is a NA value; we will need to verify the netCDF data again for
# this grid cell, possibly this is the same error in other gridcells.

write.csv(rl.cals, outfile)